In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2024-03-20 11:43:47.025335: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 11:43:47.029494: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 11:43:47.080974: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 11:43:47.992006: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-20 11:43:48.842111: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/b

[]

2024-03-20 11:43:48.842557: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
import wget, zipfile, os, random

# download dataset
if not os.path.isfile("data.zip"): 
    print("downloading...")
    wget.download("https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/3362/31148/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1678299698&Signature=b6XuKNXvsSuBcv2SZYiTqcfr7Mc%2FjpSR%2BptWRsmjRTHr1vqLdnpC6k9YaLk%2BMRSmTeZcOXGFg21TGUyv9RR7TctRcWLW8WhIdibLM2BTLqc2YCM5ZXU9DkwYRypzgmXjZs4U%2B2n4AeOzO2w7CJFnxgjOeOtAamziNd5F07aiSap1A8b2PwtClwvsQA0kx1vp0HTtT99NsWpGFY310ZOMjRnpFK17ANh5UmYAaFeHm2RVyR%2FRRVcWbVTSWKhaEDLgppgdIREZ%2FGsXpybkFAqfWGqmHkDVEpty0E3BuyGGZ8DEkDphwjHlab824Waid0IpOeOgO6qAziK5fy22dCO0Pg%3D%3D&response-content-disposition=attachment%3B+filename%3Ddogs-vs-cats.zip", "data.zip")

# extract dataset
if not (os.path.isdir("data") and os.path.isdir("data/train")): 
    print("extracting...")
    with zipfile.ZipFile("data.zip"      , 'r') as file: file.extractall("./data/")

# get all train paths
paths = list(map(lambda name:os.path.join("data/train",name), os.listdir("data/train/")))
random.shuffle(paths)
test_paths  = paths[:len(paths)//3]
train_paths = paths[len(paths)//3+1:]

In [3]:
import random, copy
def batchfy(paths, batch_size):
    #shuffle all paths
    paths = copy.deepcopy(paths)
    random.shuffle(paths)
    
    # batchfy
    size = len(paths)
    batches = size // batch_size
    batches = [paths[i*batch_size:(i+1)*batch_size] for i in range(batches)]
    
    return batches


def load_batch(paths):
    tensor_images = tf.stack([tf.image.resize(tf.io.decode_image(tf.io.read_file(path),channels=3), (256,256))/256 for path in paths])
    tensor_labels = tf.convert_to_tensor([[1,0] if path[11:14] == "cat" else [0,1] for path in paths], dtype=float)
    return tensor_images, tensor_labels


In [4]:

class ConvBlock(tf.keras.Model):

    def __init__(self, kernel_size=(3,3), filters=16, strides=(2, 2)):
        super().__init__()

        self.conv2a = tf.keras.layers.Conv2D(filters, (1, 1), strides=strides, )
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

        self.convd = tf.keras.layers.Conv2D(filters, (1, 1), strides=strides)
        self.bnd = tf.keras.layers.BatchNormalization()

    def call(self, z):
        x = tf.nn.relu(self.bn2a(self.conv2a(z)))
        x = tf.nn.relu(self.bn2b(self.conv2b(x)))
        x = self.bn2c(self.conv2c(x))

        z = self.convd(z)
        z = self.bnd(z)

        return tf.nn.relu(x + z)


In [5]:
class MyResNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2))
        self.bn_conv1 = tf.keras.layers.BatchNormalization()
        self.max_pool = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))

        self.c0 = ConvBlock()
        self.c1 = ConvBlock()
        self.c2 = ConvBlock()
        self.c3 = ConvBlock()
        self.c4 = ConvBlock()
        
        # GAP
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.lin = tf.keras.layers.Dense(2, activation="softmax")
        
    def call(self, x):
      
        x = self.conv1(x)
        x = self.bn_conv1(x)
        x = tf.nn.relu(x)
        x = self.max_pool(x)
        
        x = self.c0(x)
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        x = self.c4(x)

        x = self.lin(self.gap(x))
        
        return x

In [ ]:
batch_size = 64
epochs = 100
model = MyResNet()
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, axis=-1)

for e in range(epochs):
    cma = 0
    
    # shuffle and batch the dataset
    batched_paths = batchfy(paths, batch_size)
    
    for i, batch in enumerate(batched_paths):
        X, Y = load_batch(batch)
        
        with tf.GradientTape() as tape:
            P = model(X)
            L = loss(Y, P)
        
        A = tf.reduce_mean(tf.cast(tf.argmax(P,-1) == tf.argmax(Y,-1), float))
        cma = cma + (A-cma) / (i+1)
        G = tape.gradient(L, model.trainable_variables)
        optim.apply_gradients(zip(G, model.trainable_variables))

        print(f"\r epoch:{e}, batch:{i}/{len(batched_paths)}, loss: {L.numpy():.5f}, acc: {A.numpy():.5f}, cma: {cma:.5f}", end=" "*5)
    print()


2024-03-20 11:46:08.264451: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 50331648 exceeds 10% of free system memory.
2024-03-20 11:46:09.118248: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.
2024-03-20 11:46:09.239967: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.
2024-03-20 11:46:09.467506: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.
2024-03-20 11:46:09.846518: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 256000000 exceeds 10% of free system memory.


 epoch:0, batch:389/390, loss: 0.68166, acc: 0.59375, cma: 0.53602     
 epoch:1, batch:389/390, loss: 0.64798, acc: 0.60938, cma: 0.59808     
 epoch:2, batch:389/390, loss: 0.61035, acc: 0.64062, cma: 0.64415     
 epoch:3, batch:389/390, loss: 0.57530, acc: 0.73438, cma: 0.67352     
 epoch:4, batch:389/390, loss: 0.56810, acc: 0.71875, cma: 0.68910     
 epoch:5, batch:389/390, loss: 0.60341, acc: 0.67188, cma: 0.70809     
 epoch:6, batch:79/390, loss: 0.52935, acc: 0.73438, cma: 0.70977     